<a href="https://colab.research.google.com/github/usman3721/Fine-Tunning-LLM-on-model/blob/main/Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
emotion_data=load_dataset("dair-ai/emotion")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1429: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [3]:
print(len(emotion_data['train']),len(emotion_data["validation"]),len(emotion_data["test"]))
print(emotion_data['train'].features)
print(emotion_data['train'][10])

16000 2000 2000
{'text': Value(dtype='string', id=None), 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}
{'text': 'i feel like i have to make the suffering i m seeing mean something', 'label': 0}


In [5]:
emotion_sample=load_dataset('dair-ai/emotion',split='train[:2000]+validation[:1000]')
print(emotion_sample)

Dataset({
    features: ['text', 'label'],
    num_rows: 3000
})


In [ ]:
!pip install transformers evaluate accelerate

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer=AutoTokenizer.from_pretrained('gpt2')


In [ ]:
if tokenizer.pad_token is None:
  tokenizer.pad_token=tokenizer.eos_token


In [ ]:
def preprocess_function(examples):
  return tokenizer(examples['text'],truncation=True,padding=True)

In [ ]:
# tokenized_train_data=emotion_train.map(preprocess_function,batched=True)
# tokenized_test_data=emotion_val.map(preprocess_function,batched=True)
tokenized_data=data.map(preprocess_function,batched=True)

In [ ]:
tokenized_data['train'][0]

In [ ]:
id2label={0:'sadness',1:'joy',2:'love',3:'anger',4:'fear',5:'surprise'}
label2id={'sadness':0,"joy":1,"love":2,"anger":3,"fear":4,"surprise":5}

In [ ]:
from transformers import AutoModelForSequenceClassification

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained('gpt2',num_labels=6,id2label=id2label,label2id=label2id)


In [ ]:
model=model.to('cuda')

In [ ]:
from transformers import DataCollatorWithPadding



In [ ]:
data_collator=DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
import numpy as np
np.unique(data['train']['label'],return_counts=True)

In [ ]:
import evaluate

f1_score=evaluate.load('f1')

In [ ]:
def compute_metrics(eval_pred):
  predictions,label=eval_pred
  predictions=np.argmax(predictions,axis=1)
  return f1_score.compute(predictions=predictions,references=label,average='weighted')

In [ ]:
from transformers import TrainingArguments,Trainer

In [ ]:
model.config.pad_token_id=model.config.eos_token_id
print(model.config)

In [ ]:

training_args=TrainingArguments(
    output_dir='my_test_model',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=8,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    logging_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
    push_to_hub=False
)

In [ ]:
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

In [ ]:
trainer.train()

In [ ]:
from transformers import pipeline
cls=pipeline(task='text-classification',model='/content/my_test_model/checkpoint-640')

In [ ]:
cls("Hi! I am feeling nostalgic")

In [ ]:
text=data['test'][20]['text']
print("Text:",text)

cls(text)